# Convert "Hey Nap" ONNX to TFLite

This notebook converts your ONNX model to TFLite format for ESP32.

In [ ]:
# Install compatible versions (fixes the mapping import error)
!pip install -q "onnx==1.15.0" "onnx-tf==1.11.0" tensorflow
print("✅ Dependencies installed with compatible versions")

In [ ]:
# Upload and rename file
from google.colab import files
import os

print("📤 Upload your ONNX file...")
uploaded = files.upload()
onnx_file = list(uploaded.keys())[0]

# Rename to remove spaces/special chars
clean_name = "hey_nap.onnx"
if onnx_file != clean_name:
    os.rename(onnx_file, clean_name)
    onnx_file = clean_name
    print(f"✅ Renamed to: {clean_name}")

print(f"✅ File ready: {onnx_file}")
print(f"   Size: {os.path.getsize(onnx_file) / 1024:.1f} KB")

In [ ]:
# Convert to TFLite using onnx-tf
import onnx
import tensorflow as tf
from onnx_tf.backend import prepare
import tempfile
import os

tflite_file = "hey_nap.tflite"

print(f"🔄 Converting {onnx_file} to TFLite...")
print("\nStep 1: Loading ONNX model...")
onnx_model = onnx.load(onnx_file)
print("   ✅ ONNX model loaded")

print("\nStep 2: Converting to TensorFlow...")
tf_rep = prepare(onnx_model)
print("   ✅ TensorFlow representation created")

print("\nStep 3: Exporting to SavedModel...")
with tempfile.TemporaryDirectory() as tmp_dir:
    tf_model_path = os.path.join(tmp_dir, "saved_model")
    tf_rep.export_graph(tf_model_path)
    print("   ✅ SavedModel exported")
    
    print("\nStep 4: Converting to TFLite...")
    converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
    tflite_model = converter.convert()
    
    print("\nStep 5: Saving TFLite file...")
    with open(tflite_file, 'wb') as f:
        f.write(tflite_model)

print(f"\n✅ Conversion successful!")
print(f"   File: {tflite_file}")
print(f"   Size: {os.path.getsize(tflite_file) / 1024:.1f} KB")

In [ ]:
# Download TFLite file
from google.colab import files
import os

tflite_file = "hey_nap.tflite"

if os.path.exists(tflite_file):
    print(f"📥 Downloading {tflite_file}...")
    files.download(tflite_file)
    print(f"\n✅ Download complete!")
    print(f"\n🎉 Your TFLite model is ready for ESP32!")
    print(f"\n📝 Next steps:")
    print(f"   1. Save to your project")
    print(f"   2. Test: python3 test_hey_nap_local.py --model {tflite_file}")
    print(f"   3. Deploy: cp {tflite_file} components/openwakeword/models/")
else:
    print(f"❌ File not found: {tflite_file}")
    print("\nAvailable files:")
    !ls -la